In [1]:
from preprocessing.dependency import parent_dir
from preprocessing.common.basics import *
from preprocessing.common.save import make_dir, save_pickle, load_pickle, save
import random
import tqdm

ModuleNotFoundError: No module named 'preprocessing'

In [ ]:
def txt(v, fields, mode = 'train'):
    """ transform the recipe dict to string
    
        Args:
          v: A dict of recipe information
          fields: an order list, the last is the field to predict
          mode: string

        Returns:
          In test mode, return string X, y
          In train mode, return string X + y
    """
    to_write = ''
    for field in fields:
        if field == 'title':
            name = v['title']
            to_write += '<unused0>'+name+'<unused1>'
        if field == 'ingredients':
            ingredients = v['ingredients']
            to_write += '<unused2>'+'$'.join(ingredients)+'$<unused3>'
        if field == 'instructions':
            instructions = v['instructions']
            to_write += '<unused4>'+ instructions +'<unused5>'
    to_write += '\n'
            
    if mode == 'train':
        return to_write
                                                     
    elif mode == 'test':
        field_to_predict = '<start-%s>'%fields[-1]
        to_X, to_y = to_write.split(field_to_predict)
        return to_X+field_to_predict, to_y

class to_gpt2:
    def __init__(self, data, ls = None):
        if not ls:
            ls = list(range(len(data)))
        random.seed(42)
        random.shuffle(ls)
        #self.ls_test = ls[:2173] #10, 10, 80, test, val, train
        self.ls_val = ls[:2173] #수정, test set 없앰.
        self.ls_train = ls[2173:]
        self.data = data
        
    def test(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        # order of field and the inital of the field to predict
        orders = [(['title','ingredients','instructions'], 'd'),
                  (['title','instructions','ingredients'], 'i'),
                  (['ingredients','instructions','title'], 't')]
        
        for i, v in tqdm.tqdm(enumerate(self.data)):
            if i in ls:
                for order, init in orders:
                    X, y = txt(v, order, mode = 'test')
                    self.save(filename+'X/%d'%(i)+'%s.txt'%(init), X, overwrite)
                    self.save(filename+'y/%d'%(i)+'%s.txt'%(init), y, overwrite)
        
    def save(self, filename, to_write, overwrite = False):
        make_dir(filename)
        if os.path.isfile(filename) == True and overwrite == False:
            print('already exists'+filename)
        else:    
            with open(filename,'w') as f:
                f.write('%s' % to_write)
                
    def make_txt(self, ls, filename, tag, overwrite = False):
        make_dir(filename)
        f = open(f'{filename}/{tag}', 'w')
        for i, v in tqdm.tqdm(enumerate(self.data)):
            if i in ls:
                f.writelines(self.encode_recipe(self.data[i]))      
        print(f'{filename}/{tag}')
        f.close()
        
    def encode_recipe(self,recipe):
        return txt(recipe, ['title','ingredients','instructions'])
    
    def fast_txt(self, filename, overwrite = False):
        self.make_txt(self.ls_train, filename, tag = 'recipegpt_train.txt')
        self.make_txt(self.ls_val, filename, tag = 'recipegpt_val.txt')
        #self.make_txt(self.ls_test, filename, tag = 'recipegpt_test.txt')